# Tugas IF4070 - Representasi Pengetahuan dan Penalaran
# Implementasi Ripple Down Rules

# Simple RDR
**Simple RDR** is a ripple down rules implemented in Python.

## Setup
Assuming you've installed the latest version of Python (if not, guides for it are widely available),
1. ensure pip is installed by running `python -m ensurepip --upgrade`;
2. install the Python dependencies by running `pip install -r requirements.txt`.

In [122]:
import random

In [123]:
class Node:
    def __init__(
        self,
        precedent: str,
        antecedent: str,
        cornerstone: set[str],
        except_: "Node" = None,
        else_: "Node" = None,
        is_root: bool = False,
    ) -> None:
        self._precedent = precedent
        self._antecedent = antecedent
        self._cornerstone = cornerstone
        self._except = except_
        self._else = else_
        self._is_root = is_root

    def get_antecedent(self) -> str:
        return self._antecedent

    def get_cornerstone(self) -> set[str]:
        return self._cornerstone

    def get_except(self) -> "Node":
        return self._except

    def get_else(self) -> "Node":
        return self._else

    def set_except(self, except_: "Node") -> None:
        self._except = except_

    def set_else(self, else_: "Node") -> None:
        self._else = else_

    def match_precedent(self, case: set[str]) -> bool:
        if self._is_root:
            return True
        else:
            for statement in case:
                if statement.startswith("~"):
                    exec(f"{statement[1:]} = False")
                else:
                    exec(f"{statement} = True")

            try:
                eval(self._precedent)
                return True
            except NameError:
                return False
            finally:
                for statement in case:
                    if statement.startswith("~"):
                        exec(f"del {statement[1:]}")
                    else:
                        exec(f"del {statement}")

In [124]:
class Tree:
    def __init__(self, root: "Node"):
        self._root = root

    def _traverse_tree(self, case: set[str]) -> None:
        previous_node = self._root
        current_node = self._root
        last_true = self._root

        while current_node:
            if next_node := current_node.match_precedent(case):
                last_true = current_node
                previous_node = current_node
                current_node = current_node.get_except()
            else:
                previous_node = current_node
                current_node = current_node.get_else()

        antecedent = last_true.get_antecedent()
        print(f"The system concludes that your case can be associated with the following: {antecedent}.")
        print("Do you agree? (y/n)")

        while True:
            agreement = input(
                f"The system concludes that your case can be associated with the following: {antecedent}.\nDo you agree? (y/n)"
            ).lower()
            print(f"You entered: {agreement}")
            if agreement in ("y", "n"):
                break
            else:
                print(f"Please input a valid option!")

        if agreement == "n":
            print("Please input a correct conclusion for this case!")
            conclusion = input("Please input a correct conclusion for this case!")
            print(f"You entered: {conclusion}")

            if len(case - last_true.get_cornerstone()) == 0:
                new_precedent = random.choice(tuple(case))
            else:
                try:
                    new_precedent = random.choice(tuple(case - last_true.get_cornerstone()))
                except:
                    new_precedent = "~" + random.choice(tuple(last_true.get_cornerstone() - case))
            new_node = Node(new_precedent, conclusion, case)
            if next_node:
                previous_node.set_except(new_node)
            else:
                previous_node.set_else(new_node)
    
    def _traverse_tree_by_dataset(self, case: set[str], label: str) -> None:
        previous_node = self._root
        current_node = self._root
        last_true = self._root

        while current_node:
            if next_node := current_node.match_precedent(case):
                last_true = current_node
                previous_node = current_node
                current_node = current_node.get_except()
            else:
                previous_node = current_node
                current_node = current_node.get_else()

        if (label != last_true.get_antecedent()):
            if len(case - last_true.get_cornerstone()) == 0:
                new_precedent = random.choice(tuple(case))
            else:
                try:
                    new_precedent = random.choice(tuple(case - last_true.get_cornerstone()))
                except:
                    new_precedent = "~" + random.choice(tuple(last_true.get_cornerstone() - case))
            new_node = Node(new_precedent, label, case)
            if next_node:
                previous_node.set_except(new_node)
            else:
                previous_node.set_else(new_node)


    def start(self) -> None:
        print("Welcome to RDR Expert System!")
        print()

        while True:
            print("Enter your case, separated by a comma for each fact!")
            print("Example case: mammal, fly, ~swim")
            print("Input your case!")

            case = set(input("Input your case: ").split(", "))
            print(f"You entered: {case}")
            self._traverse_tree(case)

            while True:
                print()
                print("Would you like to evaluate a different case? (y/n)")
                continue_use = input("Would you like to evaluate a different case? (y/n) ").lower()
                if continue_use in ("y", "n"):
                    break
                else:
                    print("Please input a valid option!")

            if continue_use == "y":
                continue
            else:
                break

    def fit(self, cases: list[set[str]], labels: list[str]) -> None:
        for i in range(len(cases)):
            # print(f"Training case {i+1}: {cases[i]} -> {labels[i]}")
            self._traverse_tree_by_dataset(cases[i], labels[i])
    
    def predict(self, case: set[str]) -> str:
        current_node = self._root
        last_true = self._root

        while current_node and (current_node.get_except() or current_node.get_else()):
            if current_node.match_precedent(case):
                last_true = current_node
                current_node = current_node.get_except()
            else:
                current_node = current_node.get_else()
        else:
            if current_node.match_precedent(case):
                last_true = current_node

        return last_true.get_antecedent()
        

# Dataset yang digunakan - Zoo.csv
sumber : https://www.kaggle.com/datasets/uciml/zoo-animal-classification?select=zoo.csv

## Set up

In [125]:
import pandas as pd

df = pd.read_csv("./dataset/zoo.csv")
df.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


## Deskripsi Dataset
Dataset yang digunakan adalah dataset Zoo.csv yang berisi data mengenai 101 hewan unik yang terdiri dari 16 atribut. Atribut-atribut tersebut adalah sebagai berikut:
| Atribut | Tipe Data | Keterangan |
| --- | --- | --- |
| animal_name | String (unique) | Nama hewan |
| hair | Boolean | Apakah hewan tersebut berbulu? |
| feathers | Boolean | Apakah hewan tersebut berbulu? |
| eggs | Boolean | Apakah hewan tersebut bertelur? |
| milk | Boolean | Apakah hewan tersebut menyusui? |
| airborne | Boolean | Apakah hewan tersebut terbang? |
| aquatic | Boolean | Apakah hewan tersebut hidup di air? |
| predator | Boolean | Apakah hewan tersebut predator? |
| toothed | Boolean | Apakah hewan tersebut ber gigi? |
| backbone | Boolean | Apakah hewan tersebut memiliki tulang belakang? |
| breathes | Boolean | Apakah hewan tersebut bernafas? |
| venomous | Boolean | Apakah hewan tersebut berbisa? |
| fins | Boolean | Apakah hewan tersebut memiliki sirip? |
| legs | Integer | Jumlah kaki hewan tersebut |
| tail | Boolean | Apakah hewan tersebut memiliki ekor? |
| domestic | Boolean | Apakah hewan tersebut jinak? |
| catsize | Boolean | Apakah hewan tersebut berukuran besar? |
| class_type | Integer | Tipe kelas hewan tersebut |


## Preprocessing Dataset
preprocessing pada dataset dilakukan dengan mengabaikan kolom yang densitas datanya merupakan noise serta mengabaikan baris pada tiap kolom yang mengandung missing value.

In [126]:
df.shape

(101, 18)

In [127]:
df.isnull().sum()

animal_name    0
hair           0
feathers       0
eggs           0
milk           0
airborne       0
aquatic        0
predator       0
toothed        0
backbone       0
breathes       0
venomous       0
fins           0
legs           0
tail           0
domestic       0
catsize        0
class_type     0
dtype: int64

Dikarenakan tidak ada kolom maupun baris yang mengandung missing value, maka tidak perlu dilakukan pengabaian data. Selanjutnya, untuk simplifikasi penggunaan dataset, kolom <b>legs</b> dan <b>class_type</b> akan dihapus. <br/>
Perlu ditekankan bahwa pada dataset ini, kolom <b>animal_name</b> akan bertindak sebagai label dari data.

In [128]:
df = df[["animal_name", "hair", "feathers", "eggs", "milk", "airborne", "aquatic", "predator", "toothed", "backbone", "breathes", "venomous", "fins", "tail", "domestic", "catsize"]]
df.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,tail,domestic,catsize
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,0,0,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,1,0,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,1,0,0
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,0,0,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,1,0,1


## Implementasi RDR
Pada tahap ini, akan dilakukan 2 jenis implementasi, yaitu dengan manual (diinput oleh user) dan dengan menggunakan dataset yang telah disediakan.

### Testing - with Dataset

In [129]:
def read_dataset(df: pd.DataFrame) -> (list[set[str]], list[str]):
    dataset = []
    labels = []
    for _, row in df.iterrows():
        case = set()
        for column in df.columns:
            if row[column] == 1:
                case.add(column)
            elif type(row[column]) == str:
                labels.append(row[column])
        dataset.append(case)
    return dataset, labels

In [130]:
dataset, labels = read_dataset(df)
for data in dataset:
    print(set(data))

for label in labels:
    print(label)

{'backbone', 'milk', 'toothed', 'catsize', 'hair', 'breathes', 'predator'}
{'backbone', 'milk', 'toothed', 'catsize', 'hair', 'breathes', 'tail'}
{'backbone', 'fins', 'toothed', 'aquatic', 'eggs', 'predator', 'tail'}
{'backbone', 'milk', 'toothed', 'catsize', 'hair', 'breathes', 'predator'}
{'toothed', 'hair', 'breathes', 'tail', 'milk', 'catsize', 'backbone', 'predator'}
{'backbone', 'milk', 'toothed', 'catsize', 'hair', 'breathes', 'tail'}
{'toothed', 'hair', 'breathes', 'tail', 'domestic', 'milk', 'catsize', 'backbone'}
{'backbone', 'fins', 'toothed', 'aquatic', 'eggs', 'tail', 'domestic'}
{'backbone', 'fins', 'toothed', 'aquatic', 'eggs', 'predator', 'tail'}
{'backbone', 'milk', 'toothed', 'hair', 'breathes', 'domestic'}
{'toothed', 'hair', 'breathes', 'tail', 'milk', 'catsize', 'backbone', 'predator'}
{'backbone', 'breathes', 'feathers', 'airborne', 'eggs', 'tail', 'domestic'}
{'backbone', 'fins', 'toothed', 'aquatic', 'eggs', 'predator', 'tail'}
{'eggs', 'predator'}
{'aquatic', '

In [131]:
root_node = Node("", "human", set(), is_root=True)
model = Tree(root_node)

model.fit(dataset, labels)

In [132]:
testData = {'toothed', 'hair', 'breathes', 'milk', 'catsize', 'backbone', 'predator'}
print(model.predict(testData))

gorilla


### Testing - Manual

In [133]:
root_node = Node("", "human", set(), is_root=True)
model = Tree(root_node)

model.start()